# CSIRO Image2Biomass - Power Ensemble (Swin-V2 + ConvNeXt)

This notebook ensembles two different architectures to achieve a higher R2 score by leveraging both Transformer and CNN features.

**Models included:**
1. **Swin-V2 Base** (R2: 0.57)
2. **ConvNeXt Small** (R2: 0.50)

In [ ]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import timm
import albumentations as A
from albumentations.pytorch import ToTensorV2

# ==========================================
# CONFIGURATION
# ==========================================
DATA_DIR = "/kaggle/input/csiro-biomass"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
TARGET_COLUMNS = ['Dry_Clover_g', 'Dry_Dead_g', 'Dry_Green_g', 'GDM_g', 'Dry_Total_g']
NUM_SPECIES = 15

# WEIGHTS (Modify as you get more scores)
ENSEMBLE_WEIGHTS = {
    "swin": 0.6,    # Swin scored higher (0.57)
    "convnext": 0.4 # ConvNeXt scored 0.50
}

# PATHS (Update to your Kaggle Input paths)
SWIN_PATH = "/kaggle/input/csiro-weights/best_swin_v3.pth"
CONV_PATH = "/kaggle/input/csiro-weights/best_advanced_ConvNextSmall_fold1.pth"

In [ ]:
class AdvancedSwinHydra(nn.Module):
    def __init__(self, model_name="swinv2_base_window12_192"):
        super().__init__()
        # Strictly offline mode
        self.backbone = timm.create_model(model_name, pretrained=False, num_classes=0, img_size=(384, 768))
        embed_dim = self.backbone.num_features
        
        self.meta_reg = nn.Linear(embed_dim, 2)
        self.meta_cls = nn.Linear(embed_dim, NUM_SPECIES)
        self.species_emb = nn.Embedding(NUM_SPECIES, 32)
        
        fusion_dim = embed_dim + 2 + 32
        self.heads = nn.ModuleList([
            nn.Sequential(nn.Linear(fusion_dim, 256), nn.GELU(), nn.Linear(256, 1))
            for _ in range(5)
        ])
        
    def forward(self, x):
        feat = self.backbone(x)
        p_reg = self.meta_reg(feat)
        p_cls = self.meta_cls(feat)
        
        spec_idx = torch.argmax(p_cls, dim=1)
        s_emb = self.species_emb(spec_idx)
        
        fusion = torch.cat([feat, p_reg, s_emb], dim=1)
        out = torch.cat([h(fusion) for h in self.heads], dim=1)
        return out

class AdvancedHydraModel(nn.Module):
    def __init__(self, model_name="convnext_small"):
        super().__init__()
        # Strictly offline mode
        self.backbone = timm.create_model(model_name, pretrained=False, num_classes=0)
        embed_dim = self.backbone.num_features
        
        self.meta_reg = nn.Sequential(nn.Linear(embed_dim, 128), nn.GELU(), nn.Linear(128, 2))
        self.meta_cls = nn.Sequential(nn.Linear(embed_dim, 128), nn.GELU(), nn.Linear(128, NUM_SPECIES))
        self.species_emb = nn.Embedding(NUM_SPECIES, 32)
        
        fusion_dim = embed_dim + 2 + 32
        self.heads = nn.ModuleList([
            nn.Sequential(nn.Linear(fusion_dim, 256), nn.GELU(), nn.Linear(256, 1))
            for _ in range(5)
        ])
        
    def forward(self, x):
        feat = self.backbone(x)
        p_reg = self.meta_reg(feat)
        p_cls = self.meta_cls(feat)
        
        spec_idx = torch.argmax(p_cls, dim=1)
        s_emb = self.species_emb(spec_idx)
        
        fusion = torch.cat([feat, p_reg, s_emb], dim=1)
        out = torch.cat([h(fusion) for h in self.heads], dim=1)
        return out

In [ ]:
class EnsembleDataset(Dataset):
    def __init__(self, df, img_dir, swin_tf, conv_tf):
        self.df = df
        self.img_dir = img_dir
        self.swin_tf = swin_tf
        self.conv_tf = conv_tf
        
    def __len__(self):
        return len(self.df)
        
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(self.img_dir, row['image_path'])
        img = Image.open(img_path).convert("RGB")
        img_np = np.array(img)
        
        # Transform for each backbone type
        img_swin = self.swin_tf(image=img_np)['image']
        img_conv = self.conv_tf(image=img_np)['image']
        
        return img_swin, img_conv

In [ ]:
def run_ensemble():
    test_df = pd.read_csv(os.path.join(DATA_DIR, "test.csv"))
    unique_test = test_df.drop_duplicates(subset=['image_path']).copy()
    
    # 1. Initialize Swin
    print("Loading Swin-V2...")
    swin_model = AdvancedSwinHydra().to(DEVICE)
    if os.path.exists(SWIN_PATH):
        sd_swin = torch.load(SWIN_PATH, map_location=DEVICE)
        swin_model.load_state_dict({k.replace('module.', ''): v for k, v in sd_swin.items()})
    swin_model.eval()
    
    # 2. Initialize ConvNeXt
    print("Loading ConvNeXt...")
    conv_model = AdvancedHydraModel().to(DEVICE)
    if os.path.exists(CONV_PATH):
        sd_conv = torch.load(CONV_PATH, map_location=DEVICE)
        conv_model.load_state_dict({k.replace('module.', ''): v for k, v in sd_conv.items()})
    conv_model.eval()
    
    # 3. Transforms
    tf_swin = A.Compose([
        A.Resize(384, 768), 
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), 
        ToTensorV2()
    ])
    tf_conv = A.Compose([
        A.Resize(224, 448), 
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), 
        ToTensorV2()
    ])
    
    loader = DataLoader(EnsembleDataset(unique_test, DATA_DIR, tf_swin, tf_conv), batch_size=1, shuffle=False)
    
    results = []
    print("Starting inference loop...")
    with torch.no_grad():
        for i, (img_s, img_c) in enumerate(loader):
            # Swin Prediction
            p_s = swin_model(img_s.to(DEVICE).float()).cpu().numpy()[0]
            
            # ConvNext Prediction
            p_c = conv_model(img_c.to(DEVICE).float()).cpu().numpy()[0]
            
            # Weighted Average Blending
            p_final = (p_s * ENSEMBLE_WEIGHTS['swin']) + (p_c * ENSEMBLE_WEIGHTS['convnext'])
            
            img_path = unique_test.iloc[i]['image_path']
            for j, col in enumerate(TARGET_COLUMNS):
                results.append({
                    'image_path': img_path, 
                    'target_name': col, 
                    'target': max(0.0, float(p_final[j]))
                })
    
    # 4. Final Submission Assembly
    pred_df = pd.DataFrame(results)
    submission = test_df[['sample_id', 'image_path', 'target_name']].merge(pred_df, on=['image_path', 'target_name'], how='left')
    submission[['sample_id', 'target']].to_csv("submission.csv", index=False)
    print("\nSuccess! Submission saved to submission.csv")
    print(submission[['sample_id', 'target']].head())

if __name__ == "__main__":
    run_ensemble()